want to try and optimize feature selection

from what i can see, accident, vehicle, and person datasets seem the most relevant, might try to take some features from there

converting each csv into a dataframe for better handling
- df is our base df, will be writing it to a new csv when done
- finding intersection btwn all three csvs, using that to drop features from df1 and df2.
- i then will be adding df1 and df2 (with only unique cols) into df
- from here, determine features with strongest correlation to target since our max is 20 features
- going to run decision tree model from demo.ipynb on new csv to see if any improvements in accuracy

In [51]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
RANDOM_STATE = 42
files = ["accident23", "person23", "vehicle23"]

df = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\accident23.csv")
df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")
df2 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\person23.csv")

df3 = pd.DataFrame()

C:\Users\15013\AppData\Local\Temp\ipykernel_25672\4190137545.py:12: DtypeWarning: Columns (57,59,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")


In [52]:
accidents_col = list(df.columns)
vehicles_col = list(df1.columns)
person_col = list(df2.columns)

intersect = []

for x in accidents_col: #there has to be a better way to do this.. 
    if x in vehicles_col:
        if x in person_col:
            intersect.append(x) #if feature is in all 3 dfs

intersect.remove("CASENUM") #we wanna keep casenum since this is our joining feature
#gonna use this list to drop features from df1 and df2

df1 = df1.drop(columns = intersect, errors = "ignore")
df2 = df2.drop(columns = intersect, errors = "ignore")

print(len(df))
print(len(df1))
print(len(df2)) #each file has a differnt num of entries, we need to match case numbers

50103
87461
122388


In [53]:
#merge all three dfs based on casenum

#when i first ran all my code the df len was 500,000
#assuming this is because of multiple people per car in accident?
# if we remove these two lines then we will have much more data to work with
df1_first = df1.groupby("CASENUM").head(1)
df2_first = df2.groupby("CASENUM").head(1)

df3 = (
    df
    .merge(df1_first, on="CASENUM", how="left")
    .merge(df2_first, on="CASENUM", how="left")
)

"""df3 = (
    df
    .merge(df1, on="CASENUM", how="left")
    .merge(df2, on="CASENUM", how="left")
)"""


df3 = df3[df3['MAX_SEV'] <= 4] #drop where sev vals > 4
print(list(df3.columns))

#so by this point what we have is df3, which is a dataframe consisting of
#each unique column in the three csvs
print(len(df3))

['CASENUM', 'PSU', 'PSU_VAR', 'PSUSTRAT', 'REGION', 'REGIONNAME', 'URBANICITY', 'URBANICITYNAME', 'STRATUM', 'STRATUMNAME', 'PJ', 'WEIGHT', 'PEDS', 'PERNOTMVIT', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PERMVIT', 'MONTH', 'MONTHNAME', 'DAY_WEEK', 'DAY_WEEKNAME', 'WKDY_IM', 'WKDY_IMNAME', 'YEAR', 'YEARNAME', 'HOUR', 'HOURNAME', 'HOUR_IM', 'HOUR_IMNAME', 'MINUTE', 'MINUTENAME', 'MINUTE_IM', 'MINUTE_IMNAME', 'HARM_EV', 'HARM_EVNAME', 'EVENT1_IM', 'EVENT1_IMNAME', 'MAN_COLL', 'MAN_COLLNAME', 'MANCOL_IM', 'MANCOL_IMNAME', 'RELJCT1', 'RELJCT1NAME', 'RELJCT1_IM', 'RELJCT1_IMNAME', 'RELJCT2', 'RELJCT2NAME', 'RELJCT2_IM', 'RELJCT2_IMNAME', 'TYP_INT', 'TYP_INTNAME', 'REL_ROAD', 'REL_ROADNAME', 'WRK_ZONE', 'WRK_ZONENAME', 'LGT_COND', 'LGT_CONDNAME', 'LGTCON_IM', 'LGTCON_IMNAME', 'WEATHER', 'WEATHERNAME', 'WEATHR_IM', 'WEATHR_IMNAME', 'SCH_BUS_x', 'SCH_BUSNAME_x', 'INT_HWY', 'INT_HWYNAME', 'MAX_SEV', 'MAX_SEVNAME', 'MAXSEV_IM', 'MAXSEV_IMNAME', 'NUM_INJ', 'NUM_INJNAME', 'NO_INJ_IM', 'NO_INJ_IMNAME', '

In [54]:
#checking for missing/unknown values and changing to NaN
#most of the code taken from demo.ipynb

#tentative colnames from each csv
acc_cols = [
    "MONTH",
    "DAY_WEEK",
    "HOUR",
    "WEATHER",
    "LGT_COND",
    "HARM_EV", #first harmful event that took place
    "MAN_COLL", #the kind of collision
    "REL_ROAD",
    "TYP_INT",
    "URBANICITY",#was the area urban or rural
    "MAX_SEV" 
]

veh_cols = [
    "VSPD_LIM", #speed limit of the area
    "VSURCOND",
    "ACC_TYPE", #type of accident
    "SPEEDREL", #was accident speeding related
    "DEFORMED" #extent of damage
]

person_cols = [
    "AGE",
    "SEX",
    "PER_TYP", #role of the person at time of crash i.e. driver, occupant, etc
    "SEAT_POS", #where in car person was sitting
    "REST_USE" #restraint system used i.e. seatbelt, motorcycle helmet, etc
]

df4 = df3[acc_cols + veh_cols + person_cols]

UNKNOWN_MAP = {
    #accident csv
    "DAY_WEEK": {9},
    "HOUR": {99},
    "HARM_EV": {99},
    "MAN_COLL": {99, 98},
    "TYP_INT": {99, 98},
    "REL_ROAD": {99, 98},
    "LGT_COND": {9, 8},
    "WEATHER": {98, 99},
    
    #vehicle csv
    "VSPD_LIM": {98, 99},
    "VSURCOND": {99},
    "ACC_TYPE": {5, 10, 16, 33, 43, 49, 53, 63, 67, 75, 85, 91, 99},
    "SPEEDREL": {8, 9},
    "DEFORMED": {8,9},
    
    #person csv
    "AGE": {998, 999},
    "SEX": {8, 9},
    "PER_TYP": {9,  19},
    "SEAT_POS": {98, 99},
    "REST_USE": {98, 99}
}

rows = []
n = len(df4)
for col in df4.columns:
    codes = UNKNOWN_MAP.get(col, set())
    if len(codes) == 0:
        unk_count = 0
    else:
        unk_count = df4[col].isin(codes).sum()
    rows.append({
        "column": col,
        "unknown_codes": sorted(codes),
        "unknown_count": int(unk_count),
        "unknown_percent": round(100 * unk_count / n, 3) if n else 0.0
    })
unknown_summary = pd.DataFrame(rows).sort_values("unknown_percent", ascending=False)
print(unknown_summary)
df_clean = df4.copy()
for col, codes in UNKNOWN_MAP.items():
    if codes:
        df_clean.loc[df_clean[col].isin(codes), col] = np.nan

        column                                      unknown_codes  \
11    VSPD_LIM                                           [98, 99]   
20    REST_USE                                           [98, 99]   
16         AGE                                         [998, 999]   
15    DEFORMED                                             [8, 9]   
17         SEX                                             [8, 9]   
14    SPEEDREL                                             [8, 9]   
3      WEATHER                                           [98, 99]   
13    ACC_TYPE  [5, 10, 16, 33, 43, 49, 53, 63, 67, 75, 85, 91...   
2         HOUR                                               [99]   
4     LGT_COND                                             [8, 9]   
6     MAN_COLL                                           [98, 99]   
8      TYP_INT                                           [98, 99]   
12    VSURCOND                                               [99]   
7     REL_ROAD                    

In [55]:
#drop rows where NaN vals > 50% of the row
#for remaining rows, impute missing NaN vals
df_drop = df_clean.dropna(thresh=0.5)

imputer = SimpleImputer(strategy="median")
df_imputed = pd.DataFrame(
    imputer.fit_transform(df_drop),
    columns=df_drop.columns
)
print(df_imputed.isnull())


       MONTH  DAY_WEEK   HOUR  WEATHER  LGT_COND  HARM_EV  MAN_COLL  REL_ROAD  \
0      False     False  False    False     False    False     False     False   
1      False     False  False    False     False    False     False     False   
2      False     False  False    False     False    False     False     False   
3      False     False  False    False     False    False     False     False   
4      False     False  False    False     False    False     False     False   
...      ...       ...    ...      ...       ...      ...       ...       ...   
49058  False     False  False    False     False    False     False     False   
49059  False     False  False    False     False    False     False     False   
49060  False     False  False    False     False    False     False     False   
49061  False     False  False    False     False    False     False     False   
49062  False     False  False    False     False    False     False     False   

       TYP_INT  URBANICITY 

In [ ]:
#starting on the model
#final accuracy around 54% for only one year of data, around 49000 points
#hopefully would improve further with full dataset
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

X = df_imputed.drop(columns=['MAX_SEV'])
y = df_imputed['MAX_SEV']
for col in X.columns:
    X[col] = X[col].astype('category').cat.codes
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

dt = DecisionTreeClassifier(max_depth=20, min_samples_split= 200, random_state=RANDOM_STATE)

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Decision Tree Accuracy: 0.5499694272708744
              precision    recall  f1-score   support

         0.0       0.64      0.89      0.75      7157
         1.0       0.32      0.17      0.22      2984
         2.0       0.37      0.32      0.34      2591
         3.0       0.36      0.23      0.28      1648
         4.0       0.00      0.00      0.00       339

    accuracy                           0.55     14719
   macro avg       0.34      0.32      0.32     14719
weighted avg       0.48      0.55      0.50     14719



C:\Users\15013\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\15013\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\15013\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_clas